マウント

In [14]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [15]:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # check auth
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

データセットの読み込み

In [16]:
# downloaded = drive.CreateFile({'id':'https://drive.google.com/file/d/1G1KvlEbHKg9lT3J7iR_4hhMh0BhSG-Xi/view?usp=sharing'})

# # Download the file to a local disk as 'sample.csv'.
# downloaded.GetContentFile('/content/drive/My Drive/Colab Notebooks/Workschool/titanic_assignment_question.csv')

import pandas as pd
houseprice = pd.read_csv('drive/My Drive/Colab Notebooks/Workschool/houseprice_text.csv')
houseprice.head()

,Id,SalePrice,OverallQual,GrLivArea,GarageCars,TotRmsAbvGrd,YearBuilt,KitchenQual,ExterQual
0,1,208500,7,1710,2,8,2003,2,2
1,4,140000,7,1717,3,7,1915,2,3
2,7,307000,8,1694,2,7,2004,2,2
3,8,200000,7,2090,2,7,1973,3,3
4,9,129900,7,1774,2,8,1931,3,3


In [17]:
houseprice_pred = pd.read_csv('drive/My Drive/Colab Notebooks/Workschool/houseprice_assignment_question.csv')
houseprice_pred.head()

,Id,OverallQual,GrLivArea,GarageCars,TotRmsAbvGrd,YearBuilt,KitchenQual,ExterQual
0,2,6,1262,2,6,1976,3,3
1,3,7,1786,2,6,2001,2,2
2,5,8,2198,3,9,2000,2,2
3,6,5,1362,2,5,1993,3,3
4,10,5,1077,1,5,1939,3,3


In [18]:
import numpy as np
import pandas as pd
import sklearn

In [19]:
print(houseprice.columns)
y = houseprice["SalePrice"]
x = houseprice.drop(["Id", "SalePrice"], axis=1)

# numpy配列に変換
y_array = np.array(y)
x_array = np.array(x)

print("y_array: ", y_array.shape)
print("x_array: ", x_array.shape)

Index(['Id', 'SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars',
       'TotRmsAbvGrd', 'YearBuilt', 'KitchenQual', 'ExterQual'],
      dtype='object')
y_array:  (1168,)
x_array:  (1168, 7)


データを訓練データとテストデータに分ける

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x_array, y_array, test_size=0.4, random_state=0)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_test: ", x_test.shape)
print("y_test: ", y_test.shape)

x_train:  (700, 7)
y_train:  (700,)
x_test:  (468, 7)
y_test:  (468,)


モデルの作成

In [22]:
from sklearn.ensemble import RandomForestRegressor

In [23]:
rfr = RandomForestRegressor(random_state=0)

rfr.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [24]:
y_pred = rfr.predict(x_test)

In [25]:
# 精度を測る
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_pred, y_test))

39666.8366277747

# 用意されたデータに対して作成したモデルで予測結果を出す

In [26]:
x_test = np.array(houseprice_pred.drop(["Id"], axis=1))

y_pred = rfr.predict(x_test)
print(y_pred)
print(type(y_pred))
print(y_pred.shape)

[148680.11       216294.75       325472.98       166179.18
 101820.25       140801.         153863.39571429 148328.
 304989.13        63508.6        142465.75       293362.7
 124813.08333333 317332.13       277756.25       125464.5
 326677.08       154614.         162052.         261827.06
 463586.67        94223.4        182861.5        293902.25
 112029.33333333 205187.41666667 108265.         123765.
 207116.78       133615.66666667 173006.55833333  94839.5
 194168.25       146573.         154546.1        176059.9
 324450.86       375100.37       203740.78       145414.
 110955.4        175403.5        142539.48       120479.
 119870.         181919.02933333 143053.75       120733.3
 403942.12       101876.         334489.54       121099.6
 111536.5        135518.         133430.         161763.5
  94773.         301513.53       408208.95       225290.
 145922.         177201.         135761.75       127164.1
  96233.         189352.         114669.5        185031.25
 306315.02     

PassengerId と Survived を並べたDataFrameをCSVとして書き出す

In [27]:
houseprice_Id = houseprice_pred["Id"]
print(houseprice_Id.shape)
print(houseprice_Id)
houseprice_saleprice = pd.Series(y_pred, name="SalePrice")
print(houseprice_saleprice)

y_pred_assignment_submit = pd.concat([houseprice_Id, houseprice_saleprice], axis=1)
print(y_pred_assignment_submit)

y_pred_assignment_submit.to_csv("y_pred_assignment_submit2.csv", index=None)

(292,)
0         2
1         3
2         5
3         6
4        10
       ... 
287    1416
288    1428
289    1430
290    1451
291    1453
Name: Id, Length: 292, dtype: int64
0      148680.110000
1      216294.750000
2      325472.980000
3      166179.180000
4      101820.250000
           ...      
287    194327.580000
288    137980.000000
289    178681.040000
290    143217.000000
291    149853.857143
Name: SalePrice, Length: 292, dtype: float64
       Id      SalePrice
0       2  148680.110000
1       3  216294.750000
2       5  325472.980000
3       6  166179.180000
4      10  101820.250000
..    ...            ...
287  1416  194327.580000
288  1428  137980.000000
289  1430  178681.040000
290  1451  143217.000000
291  1453  149853.857143

[292 rows x 2 columns]
